In [33]:
!pip install --upgrade pip
!pip install sagemaker --upgrade

     |████████████████████████████████| 1.5 MB 3.2 MB/s eta 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 20.0.2
    Uninstalling pip-20.0.2:
      Successfully uninstalled pip-20.0.2
Requirement already up-to-date: sagemaker in /home/ec2-user/anaconda3/envs/pytorch_latest_p36/lib/python3.6/site-packages (2.15.4)


In [2]:
from sagemaker.local import LocalSession

sagemaker_session = LocalSession()
sagemaker_session.config = {'local': {'local_code': True}}


In [3]:
import sagemaker

sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()
prefix = 'sagemaker/DEMO-pytorch-mnist'

role = sagemaker.get_execution_role()

NoCredentialsError: Unable to locate credentials

In [ ]:
sagemaker.__version__

In [3]:
bucket

'sagemaker-us-east-2-476089912147'

In [8]:
from torchvision import datasets, transforms

datasets.MNIST('data', download=True, transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
]))

Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw


Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw



Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw


Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw
Processing...


/home/ec2-user/anaconda3/envs/pytorch_latest_p36/lib/python3.6/site-packages/torchvision/datasets/mnist.py:469: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /opt/conda/conda-bld/pytorch_1595629416375/work/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Done!


Dataset MNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.1307,), std=(0.3081,))
           )

In [9]:
inputs = sagemaker_session.upload_data(path='data', bucket=bucket, key_prefix=prefix)
print('input spec (in this case, just an S3 path): {}'.format(inputs))




input spec (in this case, just an S3 path): s3://sagemaker-us-east-2-476089912147/sagemaker/DEMO-pytorch-mnist


In [10]:
!wget https://raw.githubusercontent.com/aws/amazon-sagemaker-examples/master/sagemaker-python-sdk/pytorch_mnist/mnist.py

--2020-10-26 21:54:00--  https://raw.githubusercontent.com/aws/amazon-sagemaker-examples/master/sagemaker-python-sdk/pytorch_mnist/mnist.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.200.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.200.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8448 (8.2K) [text/plain]
Saving to: ‘mnist.py’

mnist.py            100%[===================>]   8.25K  --.-KB/s    in 0s      

2020-10-26 21:54:00 (45.0 MB/s) - ‘mnist.py’ saved [8448/8448]



In [9]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(entry_point='mnist.py',
                    role=role,
                    #framework_version='1.6.0',
                    train_instance_count=2,
                    train_instance_type='local',
                    hyperparameters={
                        'epochs': 2,
                        'backend': 'gloo'
                    },
                    image_uri='763104351884.dkr.ecr.us-east-2.amazonaws.com/pytorch-training:1.6.0-cpu-py36-ubuntu16.04',
                    env={'SAGEMAKER_REQUIREMENTS': 'requirements.txt'}
                   )

train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [7]:
inputs = 's3://sagemaker-us-east-2-476089912147/sagemaker/DEMO-pytorch-mnist'

In [ ]:
estimator.fit({'training': inputs})

2020-10-26 23:09:03 Starting - Starting the training job...
2020-10-26 23:09:04 Starting - Launching requested ML instances......
2020-10-26 23:10:16 Starting - Preparing the instances for training......
2020-10-26 23:11:17 Downloading - Downloading input data...
2020-10-26 23:11:55 Training - Downloading the training image..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2020-10-26 23:12:08,503 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2020-10-26 23:12:08,505 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2020-10-26 23:12:08,515 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2020-10-26 23:12:11,549 sagemaker_pytorch_container.training INFO     Invoking user training script.
2020-10-26 23:12:11,902 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2020-10-26 23:12:1

In [ ]:
estimator.delete_endpoint()